# Data Testing dan Training dari Artificial Neural Network dengan Perhitungan Back-Propagation

Pertama, kita import modul yang dibutuhkan dan me-load dataset yang digunakan

In [1]:
import numpy as np
from sklearn.model_selection import train_test_split

db = np.loadtxt("duke-breast-cancer.txt")
print("Database raw shape (%s,%s)" % np.shape(db))

Database raw shape (86,7130)


In [2]:
db

array([[ 1.      , -0.362772, -0.314085, ...,  0.343084, -1.24348 ,
         1.80402 ],
       [ 1.      , -0.45958 , -0.719855, ..., -0.729734, -0.639368,
         0.344724],
       [ 1.      ,  0.103909, -0.296076, ...,  0.1812  , -0.626812,
         0.237308],
       ...,
       [ 0.      , -0.512385, -0.326583, ...,  0.180162,  0.701266,
        -0.3054  ],
       [ 0.      , -0.213418,  0.415821, ...,  1.0016  , -0.390027,
        -0.62477 ],
       [ 0.      , -0.72451 , -0.359479, ...,  0.222557, -0.733495,
        -0.821411]])

In [3]:
np.random.shuffle(db)
y = db[:, 0]
x = np.delete(db, [0], axis=1)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1)
print(np.shape(x_train),np.shape(x_test))

(77, 7129) (9, 7129)


Lalu kemudian kita membuat vector hidden-layer,  weight's matrix, output layer vector dan hidden weight's matrix

In [4]:
hidden_layer = np.zeros(72)
weights = np.random.random((len(x[0]), 72))
output_layer = np.zeros(2)
hidden_weights = np.random.random((72, 2))

kemudian kita perlu mengimplementasikan fungsi sum, fungsi aktifasi, fungsi softmax, fungsi recalculate-weight dan terakhir fungsi back-propagation

-Fungsi Sumnya

In [5]:
def sum_function(weights, index_locked_col, x):
    result = 0
    for i in range(0, len(x)):
        result += x[i] * weights[i][index_locked_col]
    return result

-Fungsi Aktifasinya

In [6]:
def activate_layer(layer, weights, x):
    for i in range(0, len(layer)):
        layer[i] = 1.7159 * np.tanh(2.0 * sum_function(weights, i, x) / 3.0)

-Fungsi Softmaxnya

In [7]:
def soft_max(layer):
    soft_max_output_layer = np.zeros(len(layer))
    for i in range(0, len(layer)):
        denominator = 0
        for j in range(0, len(layer)):
            denominator += np.exp(layer[j] - np.max(layer))
        soft_max_output_layer[i] = np.exp(layer[i] - np.max(layer)) / denominator
    return soft_max_output_layer

-Fungsi Recalculate-Weight-nya

In [8]:
def recalculate_weights(learning_rate, weights, gradient, activation):
    for i in range(0, len(weights)):
        for j in range(0, len(weights[i])):
            weights[i][j] = (learning_rate * gradient[j] * activation[i]) + weights[i][j]

Dan terakhir, fungsi Back-Propagationnya

In [9]:
def back_propagation(hidden_layer, output_layer, one_hot_encoding, learning_rate, x):
    output_derivative = np.zeros(2)
    output_gradient = np.zeros(2)
    for i in range(0, len(output_layer)):
        output_derivative[i] = (1.0 - output_layer[i]) * output_layer[i]
    for i in range(0, len(output_layer)):
        output_gradient[i] = output_derivative[i] * (one_hot_encoding[i] - output_layer[i])
    hidden_derivative = np.zeros(72)
    hidden_gradient = np.zeros(72)
    for i in range(0, len(hidden_layer)):
        hidden_derivative[i] = (1.0 - hidden_layer[i]) * (1.0 + hidden_layer[i])
    for i in range(0, len(hidden_layer)):
        sum_ = 0
        for j in range(0, len(output_gradient)):
            sum_ += output_gradient[j] * hidden_weights[i][j]
        hidden_gradient[i] = sum_ * hidden_derivative[i]
    recalculate_weights(learning_rate, hidden_weights, output_gradient, hidden_layer)
    recalculate_weights(learning_rate, weights, hidden_gradient, x)

Kemudian kita masuk ke dalam training, dan terakhir adalah testing,
Dimana dibawah ini adalah code untuk testing

In [10]:
one_hot_encoding = np.zeros((2,2))
for i in range(0, len(one_hot_encoding)):
    one_hot_encoding[i][i] = 1
training_correct_answers = 0
for i in range(0, len(x_train)):
    activate_layer(hidden_layer, weights, x_train[i])
    activate_layer(output_layer, hidden_weights, hidden_layer)
    output_layer = soft_max(output_layer)
    training_correct_answers += 1 if y_train[i] == np.argmax(output_layer) else 0
    back_propagation(hidden_layer, output_layer, one_hot_encoding[int(y_train[i])], -1, x_train[i])
print("MLP Correct answers while learning: %s / %s (Accuracy = %s) on %s database." % (training_correct_answers, len(x_train), 
                                                                                       training_correct_answers/len(x_train),"Duke breast cancer"))

MLP Correct answers while learning: 52 / 77 (Accuracy = 0.6753246753246753) on Duke breast cancer database.


Setelah selesai proses training, dengan hasil 0.0.6753246753246753,
Kita melakukan proses testing.

In [11]:
testing_correct_answers = 0
for i in range(0, len(x_test)):
    activate_layer(hidden_layer, weights, x_test[i])
    activate_layer(output_layer, hidden_weights, hidden_layer)
    output_layer = soft_max(output_layer)
    testing_correct_answers += 1 if y_test[i] == np.argmax(output_layer) else 0
print("MLP Correct answers while testing: %s / %s (Accuracy = %s) on %s database" % (testing_correct_answers, len(x_test),
                                                                                     testing_correct_answers/len(x_test), "Duke breast cancer"))

MLP Correct answers while testing: 8 / 9 (Accuracy = 0.8888888888888888) on Duke breast cancer database


Proses testing slesai dengan hasil akurasi 0.7777777777777778, dimana dari hasil ini dapat kita simpulkan bahwa, jumlah error yang didapatkan masih dalam angka yang besar
Karena hasilnya masih jauh dari hasil sempurna, 1,

Sumber : https://www.kaggle.com/andreicosma/back-propagation-neural-network